# Notebook to create a standard load profile based on most recent data available

In [18]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [19]:
dynamic_factors = pd.read_csv('/Users/juliusbecker/Project_Energy_Tariffs/app_data/standard_profile/dynamic_factors.csv')
energy_usage = pd.read_csv('/Users/juliusbecker/Project_Energy_Tariffs/app_data/standard_profile/energy_usage.csv')

In [20]:
# Let's examine the structure of both files
print("Dynamic factors shape:", dynamic_factors.shape)
print("Dynamic factors columns:", dynamic_factors.columns.tolist())
print("\nFirst few rows of dynamic factors:")
print(dynamic_factors.head())

print("\n" + "="*50)
print("Energy usage shape:", energy_usage.shape)
print("Energy usage columns:", energy_usage.columns.tolist())
print("\nFirst few rows of energy usage:")
print(energy_usage.head())

Dynamic factors shape: (365, 3)
Dynamic factors columns: ['day-number', 'date', 'dynamic factor']

First few rows of dynamic factors:
   day-number   date  dynamic factor
0           1  1-Jan        1.242030
1           2  2-Jan        1.243922
2           3  3-Jan        1.245678
3           4  4-Jan        1.247299
4           5  5-Jan        1.248787

Energy usage shape: (97, 37)
Energy usage columns: ['Unnamed: 0', 'January', 'January.1', 'January.2', 'February', 'February.1', 'February.2', 'March', 'March.1', 'March.2', 'April', 'April.1', 'April.2', 'May', 'May.1', 'May.2', 'June', 'June.1', 'June.2', 'July', 'July.1', 'July.2', 'August', 'August.1', 'August.2', 'September', 'September.1', 'September.2', 'October', 'October.1', 'October.2', 'November', 'November.1', 'November.2', 'December', 'December.1', 'December.2']

First few rows of energy usage:
    Unnamed: 0 January January.1 January.2 February February.1 February.2  \
0          NaN      SA        FT        WT       SA  

In [21]:
# Clean up the energy usage data
# First row contains the day type codes (SA, FT, WT), second row starts the actual data
day_types = energy_usage.iloc[0, 1:].values  # Get SA, FT, WT codes
energy_data = energy_usage.iloc[1:, :].copy()  # Get actual usage data
energy_data.columns = energy_usage.columns  # Keep original column names

# Set the time intervals as index
energy_data.set_index('Unnamed: 0', inplace=True)

print("Day types:", day_types)
print("Energy data shape:", energy_data.shape)
print("Energy data index (time intervals):")
print(energy_data.index[:10].tolist())

Day types: ['SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT'
 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT' 'SA'
 'FT' 'WT' 'SA' 'FT' 'WT' 'SA' 'FT' 'WT']
Energy data shape: (96, 36)
Energy data index (time intervals):
['00:00-00:15', '00:15-00:30', '00:30-00:45', '00:45-01:00', '01:00-01:15', '01:15-01:30', '01:30-01:45', '01:45-02:00', '02:00-02:15', '02:15-02:30']


In [22]:
# Create datetime range from Jan 1, 2025 to end of 2026 in 15-minute intervals
start_date = datetime(2025, 1, 1)
end_date = datetime(2027, 1, 1)  # Go to start of 2027 to include all of 2026
datetime_range = pd.date_range(start=start_date, end=end_date, freq='15min', inclusive='left')

print(f"Total datetime points: {len(datetime_range)}")
print(f"First few dates: {datetime_range[:5]}")
print(f"Last few dates: {datetime_range[-5:]}")

# Verify we have 96 time intervals per day (24 hours * 4 intervals per hour)
print(f"Intervals per day: 24 hours * 4 = {24*4}")
print(f"Energy data has {len(energy_data)} time intervals per day")

Total datetime points: 70080
First few dates: DatetimeIndex(['2025-01-01 00:00:00', '2025-01-01 00:15:00',
               '2025-01-01 00:30:00', '2025-01-01 00:45:00',
               '2025-01-01 01:00:00'],
              dtype='datetime64[ns]', freq='15T')
Last few dates: DatetimeIndex(['2026-12-31 22:45:00', '2026-12-31 23:00:00',
               '2026-12-31 23:15:00', '2026-12-31 23:30:00',
               '2026-12-31 23:45:00'],
              dtype='datetime64[ns]', freq='15T')
Intervals per day: 24 hours * 4 = 96
Energy data has 96 time intervals per day


In [23]:
# Let's check the actual column structure after removing the first column
print("Energy data columns:", len(energy_data.columns))
print("Actual columns:", energy_data.columns.tolist())

# The energy data has 36 columns (excluding the index), which is 12 months * 3 day types = 36
# But we need to account for the fact that the first column (Unnamed: 0) is now the index
month_cols = {}
for i, month in enumerate(months):
    month_cols[month] = {
        'SA': energy_data.columns[i*3],      # Saturday  
        'FT': energy_data.columns[i*3 + 1],  # Sunday/Holiday (FT = Feiertag)
        'WT': energy_data.columns[i*3 + 2]   # Weekday (WT = Werktag)
    }

print("Month column mapping example for January:")
print(month_cols['January'])

Energy data columns: 36
Actual columns: ['January', 'January.1', 'January.2', 'February', 'February.1', 'February.2', 'March', 'March.1', 'March.2', 'April', 'April.1', 'April.2', 'May', 'May.1', 'May.2', 'June', 'June.1', 'June.2', 'July', 'July.1', 'July.2', 'August', 'August.1', 'August.2', 'September', 'September.1', 'September.2', 'October', 'October.1', 'October.2', 'November', 'November.1', 'November.2', 'December', 'December.1', 'December.2']


NameError: name 'months' is not defined

In [24]:
# Function to determine day type (SA=Saturday, FT=Sunday, WT=Weekday)
def get_day_type(date):
    weekday = date.weekday()  # Monday=0, Sunday=6
    if weekday == 5:  # Saturday
        return 'SA'
    elif weekday == 6:  # Sunday
        return 'FT'
    else:  # Monday-Friday
        return 'WT'

# Test the function
test_date = datetime(2025, 1, 1)  # Wednesday
print(f"Jan 1, 2025 ({test_date.strftime('%A')}) -> {get_day_type(test_date)}")

# Check day of year mapping in dynamic factors
print(f"Dynamic factors covers {len(dynamic_factors)} days")
print("Sample dynamic factors:")
print(dynamic_factors.head(10))

Jan 1, 2025 (Wednesday) -> WT
Dynamic factors covers 365 days
Sample dynamic factors:
   day-number    date  dynamic factor
0           1   1-Jan        1.242030
1           2   2-Jan        1.243922
2           3   3-Jan        1.245678
3           4   4-Jan        1.247299
4           5   5-Jan        1.248787
5           6   6-Jan        1.250145
6           7   7-Jan        1.251374
7           8   8-Jan        1.252476
8           9   9-Jan        1.253453
9          10  10-Jan        1.254306


In [25]:
# Create a sample H0 profile from the energy usage data
# Let's use January weekday data as an example

# Create time column from the index
time_intervals = energy_usage.iloc[1:, 0].tolist()  # Skip the first row (headers)
print(f"Time intervals: {len(time_intervals)}")
print(f"Sample intervals: {time_intervals[:5]}")

# Get weekday consumption values for January (January.2 column)
january_weekday_values = energy_usage.iloc[1:]['January.2'].tolist()  # Skip header row
print(f"January weekday values: {len(january_weekday_values)}")
print(f"Sample values (string): {january_weekday_values[:5]}")

# Create a DataFrame for the H0 profile
h0_profile = pd.DataFrame({
    'time': time_intervals,
    'value': pd.to_numeric(january_weekday_values)  # Convert strings to numbers
})

print("Original H0 profile (15-minute intervals):")
print(f"Original data shape: {h0_profile.shape}")
print(f"Total consumption (15-min): {h0_profile['value'].sum():.2f}")
print(f"Sample numeric values: {h0_profile['value'].head().tolist()}")

# Convert time to proper datetime format and extract hour/minute
def parse_time_interval(time_str):
    # Convert "00:00-00:15" to "00:00"
    start_time = time_str.split('-')[0]
    return start_time

h0_profile['time_parsed'] = h0_profile['time'].apply(parse_time_interval)
h0_profile['datetime'] = pd.to_datetime(h0_profile['time_parsed'], format='%H:%M')

# Resample to hourly sums (sum the 4 15-minute intervals in each hour)
h0_profile_hourly = h0_profile.groupby(h0_profile['datetime'].dt.hour)['value'].sum().reset_index()
h0_profile_hourly.columns = ['hour', 'value']

print(f"\nAfter resampling to hourly:")
print(f"Hourly data shape: {h0_profile_hourly.shape}")
print(f"Total consumption (hourly): {h0_profile_hourly['value'].sum():.2f}")
print(f"Sample hourly values: {h0_profile_hourly['value'].head().tolist()}")

# Calculate scaling factor for typical household consumption
typical_household_annual = 3500  # kWh per year
current_total = h0_profile_hourly['value'].sum()
scale_factor = typical_household_annual / (current_total * 365)  # Scale for full year

print(f"\nScaling analysis:")
print(f"Current daily total consumption: {current_total:.2f}")
print(f"Current annual (x365): {current_total * 365:.2f}")
print(f"Target annual consumption: {typical_household_annual} kWh")
print(f"Required scale factor: {scale_factor:.8f}")

# Apply scaling
h0_profile_hourly['value_scaled'] = h0_profile_hourly['value'] * scale_factor
print(f"After scaling - daily total: {h0_profile_hourly['value_scaled'].sum():.2f} kWh")
print(f"After scaling - annual estimate: {h0_profile_hourly['value_scaled'].sum() * 365:.0f} kWh")
print(f"Sample scaled hourly values: {h0_profile_hourly['value_scaled'].head().tolist()}")

# Verify realistic consumption levels
avg_hourly = h0_profile_hourly['value_scaled'].mean()
max_hourly = h0_profile_hourly['value_scaled'].max()
min_hourly = h0_profile_hourly['value_scaled'].min()

print(f"\nRealistic consumption check:")
print(f"Average hourly consumption: {avg_hourly:.4f} kWh")
print(f"Max hourly consumption: {max_hourly:.4f} kWh")
print(f"Min hourly consumption: {min_hourly:.4f} kWh")
print(f"Daily total: {avg_hourly * 24:.2f} kWh/day")
print(f"Annual total: {avg_hourly * 24 * 365:.0f} kWh/year")

Time intervals: 96
Sample intervals: ['00:00-00:15', '00:15-00:30', '00:30-00:45', '00:45-01:00', '01:00-01:15']
January weekday values: 96
Sample values (string): ['20.126', '18.915', '17.959', '17.202', '16.612']
Original H0 profile (15-minute intervals):
Original data shape: (96, 2)
Total consumption (15-min): 2476.45
Sample numeric values: [20.126, 18.915, 17.959, 17.202, 16.612]

After resampling to hourly:
Hourly data shape: (24, 2)
Total consumption (hourly): 2476.45
Sample hourly values: [74.202, 63.785, 60.389, 59.857, 62.479]

Scaling analysis:
Current daily total consumption: 2476.45
Current annual (x365): 903904.25
Target annual consumption: 3500 kWh
Required scale factor: 0.00387209
After scaling - daily total: 9.59 kWh
After scaling - annual estimate: 3500 kWh
Sample scaled hourly values: [0.28731693650074114, 0.24698135892158934, 0.23383173604947652, 0.23177178334984047, 0.2419244073694753]

Realistic consumption check:
Average hourly consumption: 0.3995 kWh
Max hourly c

In [26]:
# Now let's create a function that can be used in the EnergyTariff class
def load_scaled_standard_profile(annual_kwh=3500):
    """
    Load and scale the standard load profile to match target annual consumption.
    
    Args:
        annual_kwh: Target annual consumption in kWh
        
    Returns:
        pd.DataFrame: Hourly consumption profile scaled to target annual consumption
    """
    # Load the standard profile data
    energy_usage = pd.read_csv('/Users/juliusbecker/Project_Energy_Tariffs/app_data/standard_profile/energy_usage.csv')
    
    # Extract time intervals and January weekday consumption
    time_intervals = energy_usage.iloc[1:, 0].tolist()  # Skip header row
    january_weekday_values = pd.to_numeric(energy_usage.iloc[1:]['January.2'].tolist())
    
    # Create 15-minute profile
    profile_15min = pd.DataFrame({
        'time': time_intervals,
        'value': january_weekday_values
    })
    
    # Parse time and convert to datetime
    def parse_time_interval(time_str):
        return time_str.split('-')[0]
    
    profile_15min['time_parsed'] = profile_15min['time'].apply(parse_time_interval)
    profile_15min['datetime'] = pd.to_datetime(profile_15min['time_parsed'], format='%H:%M')
    
    # Resample to hourly
    profile_hourly = profile_15min.groupby(profile_15min['datetime'].dt.hour)['value'].sum().reset_index()
    profile_hourly.columns = ['hour', 'value']
    
    # Calculate scaling factor
    current_daily_total = profile_hourly['value'].sum()
    target_daily_consumption = annual_kwh / 365  # Target daily consumption
    scale_factor = target_daily_consumption / current_daily_total
    
    # Apply scaling
    profile_hourly['value_scaled'] = profile_hourly['value'] * scale_factor
    
    print(f"Loaded standard profile:")
    print(f"- Target annual consumption: {annual_kwh} kWh")
    print(f"- Scaling factor: {scale_factor:.8f}")
    print(f"- Daily total after scaling: {profile_hourly['value_scaled'].sum():.2f} kWh")
    print(f"- Annual total: {profile_hourly['value_scaled'].sum() * 365:.0f} kWh")
    
    return profile_hourly

# Test the function
test_profile = load_scaled_standard_profile(3500)
print("\nTest profile sample:")
print(test_profile.head())

Loaded standard profile:
- Target annual consumption: 3500 kWh
- Scaling factor: 0.00387209
- Daily total after scaling: 9.59 kWh
- Annual total: 3500 kWh

Test profile sample:
   hour   value  value_scaled
0     0  74.202      0.287317
1     1  63.785      0.246981
2     2  60.389      0.233832
3     3  59.857      0.231772
4     4  62.479      0.241924


In [27]:
# Create the standard load profile
def create_standard_load_profile():
    profile_data = []
    
    for dt in datetime_range:
        # Get the day of year (1-365 for non-leap years, 1-366 for leap years)
        day_of_year = dt.timetuple().tm_yday
        
        # For 2025 and 2026, we need to map to the 365-day dynamic factors cycle
        # Since dynamic_factors only has 365 days, we use modulo for leap years
        factor_index = (day_of_year - 1) % len(dynamic_factors)
        dynamic_factor = dynamic_factors.iloc[factor_index]['dynamic factor']
        
        # Get month name
        month_name = dt.strftime('%B')
        
        # Get day type
        day_type = get_day_type(dt)
        
        # Get the appropriate column for this month and day type
        col_name = month_cols[month_name][day_type]
        
        # Get time interval (00:00-00:15 format)
        time_str = dt.strftime('%H:%M') + '-' + (dt + timedelta(minutes=15)).strftime('%H:%M')
        
        # Find the matching row in energy_data for this time interval
        try:
            base_usage = float(energy_data.loc[time_str, col_name])
            # Apply dynamic factor
            final_usage = base_usage * dynamic_factor
            
            profile_data.append({
                'datetime': dt,
                'value': final_usage
            })
        except KeyError:
            # Handle edge case for 23:45-00:00 which might be represented differently
            if time_str == '23:45-00:00':
                time_str = '23:45-24:00'
                try:
                    base_usage = float(energy_data.loc[time_str, col_name])
                    final_usage = base_usage * dynamic_factor
                    profile_data.append({
                        'datetime': dt,
                        'value': final_usage
                    })
                except KeyError:
                    print(f"Warning: Could not find time interval {time_str}")
                    continue
            else:
                print(f"Warning: Could not find time interval {time_str}")
                continue
    
    return pd.DataFrame(profile_data)

# Generate the profile (this might take a moment)
print("Generating standard load profile...")
profile_df = create_standard_load_profile()
print(f"Generated {len(profile_df)} data points")
print("\nFirst few rows:")
print(profile_df.head(10))

Generating standard load profile...


KeyError: 'January'

In [28]:
# Verify the data
print("Profile summary:")
print(f"Date range: {profile_df['datetime'].min()} to {profile_df['datetime'].max()}")
print(f"Total data points: {len(profile_df)}")
print(f"Expected data points: {2 * 365 * 96 + 96} (2 years * 365 days * 96 intervals + 96 for leap day)")
print(f"Value range: {profile_df['value'].min():.2f} to {profile_df['value'].max():.2f}")
print(f"Average value: {profile_df['value'].mean():.2f}")

# Check for any missing data
print(f"\nMissing values: {profile_df.isnull().sum().sum()}")

# Sample data from different periods
print("\nSample data from different time periods:")
print("January 1, 2025 (Wednesday):")
jan_1_data = profile_df[profile_df['datetime'].dt.date == datetime(2025, 1, 1).date()]
print(jan_1_data.head(8))

print("\nJanuary 4, 2025 (Saturday):")
jan_4_data = profile_df[profile_df['datetime'].dt.date == datetime(2025, 1, 4).date()]
print(jan_4_data.head(4))

print("\nJanuary 5, 2025 (Sunday):")
jan_5_data = profile_df[profile_df['datetime'].dt.date == datetime(2025, 1, 5).date()]
print(jan_5_data.head(4))

Profile summary:


NameError: name 'profile_df' is not defined

In [29]:
# Save the standard load profile to CSV
output_file = '/Users/juliusbecker/Project_Energy_Tariffs/data/Standard_Load_Profile_2025_2026.csv'
profile_df.to_csv(output_file, index=False)

print(f"Standard load profile saved to: {output_file}")
print(f"File contains {len(profile_df)} rows with datetime and value columns")

# Display final summary
print("\nFinal verification:")
print(f"Data covers: {profile_df['datetime'].dt.date.min()} to {profile_df['datetime'].dt.date.max()}")
print(f"Time intervals: 15-minute intervals from 00:00 to 23:45 each day")
print(f"Total days covered: {profile_df['datetime'].dt.date.nunique()}")
print(f"Unique months: {sorted(profile_df['datetime'].dt.month.unique())}")
print(f"Value statistics:")
print(f"  Min: {profile_df['value'].min():.2f}")
print(f"  Max: {profile_df['value'].max():.2f}")
print(f"  Mean: {profile_df['value'].mean():.2f}")
print(f"  Median: {profile_df['value'].median():.2f}")

# Show sample of the saved data
print(f"\nSample of saved data:")
print(profile_df.head())

NameError: name 'profile_df' is not defined

In [30]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.patches import Rectangle

# Create daily aggregations for better visualization
daily_profile = profile_df.copy()
daily_profile['date'] = daily_profile['datetime'].dt.date
daily_profile['day_of_week'] = daily_profile['datetime'].dt.day_name()
daily_profile['month'] = daily_profile['datetime'].dt.month_name()

# Calculate daily totals
daily_totals = daily_profile.groupby('date').agg({
    'value': 'sum',
    'day_of_week': 'first',
    'month': 'first'
}).reset_index()

daily_totals['date'] = pd.to_datetime(daily_totals['date'])

print(f"Daily totals calculated for {len(daily_totals)} days")
print(f"Daily consumption range: {daily_totals['value'].min():.1f} to {daily_totals['value'].max():.1f}")
print(f"Average daily consumption: {daily_totals['value'].mean():.1f}")

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Standard Load Profile Analysis (2025-2026)', fontsize=16, fontweight='bold')

# Plot 1: Time series of daily totals
axes[0, 0].plot(daily_totals['date'], daily_totals['value'], linewidth=0.8, alpha=0.7, color='blue')
axes[0, 0].set_title('Daily Energy Consumption Over Time', fontweight='bold')
axes[0, 0].set_xlabel('Date')
axes[0, 0].set_ylabel('Daily Total Energy')
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].xaxis.set_major_locator(mdates.MonthLocator(interval=3))
axes[0, 0].xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Monthly averages comparison
monthly_avg = daily_totals.groupby(daily_totals['date'].dt.month)['value'].mean()
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
colors = ['lightblue' if i < 6 else 'lightcoral' for i in range(12)]
axes[0, 1].bar(range(1, 13), monthly_avg.values, color=colors, alpha=0.7, edgecolor='black')
axes[0, 1].set_title('Average Daily Consumption by Month', fontweight='bold')
axes[0, 1].set_xlabel('Month')
axes[0, 1].set_ylabel('Average Daily Energy')
axes[0, 1].set_xticks(range(1, 13))
axes[0, 1].set_xticklabels(month_names)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Plot 3: Day of week patterns
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_avg = daily_totals.groupby('day_of_week')['value'].mean().reindex(day_order)
weekend_colors = ['lightgreen' if day in ['Saturday', 'Sunday'] else 'lightblue' for day in day_order]
axes[1, 0].bar(range(7), dow_avg.values, color=weekend_colors, alpha=0.7, edgecolor='black')
axes[1, 0].set_title('Average Daily Consumption by Day of Week', fontweight='bold')
axes[1, 0].set_xlabel('Day of Week')
axes[1, 0].set_ylabel('Average Daily Energy')
axes[1, 0].set_xticks(range(7))
axes[1, 0].set_xticklabels([day[:3] for day in day_order])
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Plot 4: Distribution histogram
axes[1, 1].hist(daily_totals['value'], bins=30, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].axvline(daily_totals['value'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1, 1].axvline(daily_totals['value'].median(), color='orange', linestyle='--', linewidth=2, label='Median')
axes[1, 1].set_title('Distribution of Daily Energy Consumption', fontweight='bold')
axes[1, 1].set_xlabel('Daily Total Energy')
axes[1, 1].set_ylabel('Frequency (Days)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

NameError: name 'profile_df' is not defined